In [60]:
from pathlib import Path
import numpy as np
import re
from math import prod
from collections import defaultdict, deque
from copy import copy
from itertools import pairwise, permutations, product, combinations

In [196]:
data = Path('../Data/Day24.txt').read_text().splitlines()

In [197]:
wires = {}
gates = {}

for line in data:
    if line.find('->') != -1:
        w1, op, w2, _, out = line.split()
        gates[out] = [w1, w2, op]
    elif line.find(': ') != -1:
        w, v = line.split(': ')
        wires[w] = int(v)


In [243]:
def get_wire_output(wire, wires = wires, gates = gates):
    if wire in wires:
        return wires[wire]
    
    w1, w2, op = gates[wire]

    out1 = get_wire_output(w1, wires, gates)
    out2 = get_wire_output(w2, wires, gates)

    match op:
        case 'OR':
            wires[wire] = out1 | out2 
        case 'XOR':
            wires[wire] = out1 ^ out2
        case 'AND':
            wires[wire] = out1 * out2
    
    return wires[wire]

def calculate_final_output(wires = wires, gates = gates):

    for wire in gates:
        get_wire_output(wire, wires, gates)

    return int(''.join([str(wires[z]) for z in sorted([w for w in wires if w.startswith('z')], reverse = True)]), 2)

In [244]:
calculate_final_output()

57344080719736

In [174]:
num_bits = len([wire for wire in wires if wire[0] == 'x'])

def calculate_from_xy(x, y, num_bits = num_bits):
    wires2 = {}
    for i in range(num_bits):
        wires2[f"x{i:02}"] = (x >> i) % 2
        wires2[f"y{i:02}"] = (y >> i) % 2

    return calculate_final_output(wires2, gates)

In [246]:
for i in range(num_bits):
    clean_wires = {f"{x}{bit:02}": 0 for x in ['x', 'y'] for bit in range(num_bits)}
    clean_wires[f"x{i:02}"] = 1
    clean_wires[f"y{i:02}"] = 1

    if get_wire_output(f"z{i:02}", clean_wires, gates) != 0:
        print(f"z{i:02}")

z05
z15
z23


In [259]:
def traceback(wire, gates = gates):

    visited = set()

    nodes = deque([wire])

    while nodes:
        wire = nodes.pop()

        visited.add(wire)

        if wire[0] not in 'xy':
            w1, w2, op = gates[wire]

            if w1 not in visited:
                nodes.append(w1)
            if w2 not in visited:
                nodes.append(w2)

    return visited
    

In [277]:
{k:v for k,v in gates.items() if 'x05' in v and 'y05' in v}

{'svm': ['y05', 'x05', 'AND'], 'nbc': ['y05', 'x05', 'XOR']}

In [278]:
{k:v for k,v in gates.items() if 'svm' in v}

{'z05': ['rgq', 'svm', 'XOR'], 'skn': ['svm', 'rgq', 'AND']}

In [283]:
{k:v for k,v in gates.items() if 'cqp' in v}

{'z07': ['rnc', 'cqp', 'XOR'], 'hdk': ['cqp', 'rnc', 'AND']}